In [0]:
CREATE STREAMING TABLE raw_franchises
TBLPROPERTIES ("quality" = "bronze")
COMMENT "A streaming table for franchise data."
AS SELECT
  *
FROM STREAM(bakehouse.sales.franchises)

In [0]:
CREATE STREAMING TABLE raw_sales_tx
TBLPROPERTIES ("quality" = "bronze")
COMMENT "A streaming table for sales transactions."
AS SELECT
  *
FROM STREAM(bakehouse.sales.transactions)

In [0]:
CREATE MATERIALIZED VIEW flagship_locations
(CONSTRAINT supplierId_set EXPECT (f.supplierId IS NOT NULL) ON VIOLATION DROP ROW,
 CONSTRAINT correct_city EXPECT (f.city != "test") ON VIOLATION FAIL UPDATE)
TBLPROPERTIES ("quality" = "silver")
COMMENT "Silver-layered materialized view to store flagship locations."
AS SELECT
  SUM(s.totalPrice) AS total_sales,
  f.district,
  f.city,
  f.country, 
  f.supplierId
FROM
  LIVE.raw_sales_tx s
  JOIN LIVE.raw_franchises f ON s.franchiseID = f.franchiseID
WHERE
  DATE(s.dateTime) = (SELECT MAX(DATE(dateTime)) FROM LIVE.raw_sales_tx)
GROUP BY
  ALL
ORDER BY
  total_sales DESC